# Dog App preparation notebook

This notebook generated the breed dictionary and uses the saved trained model, and use the functions to predict a dog breed.


In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

Using TensorFlow backend.


## Generating the classes dictionary to names

In [2]:
# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("../../../data/dog_images/train/*/"))]
print('There are %d total dog categories.' % len(dog_names))

There are 133 total dog categories.


In [3]:
dog_names[94].split('.')[1]

'Kuvasz'

In [7]:
# define function to load train, test, and validation datasets
path='../../../data/dog_images/train'
data = load_files(path)
dog_files = np.array(data['filenames'])

unordered_dict = {}
for i in range(len(dog_files)):
    file_name = dog_files[i].split('train/')[1]
    class_number = data['target'][i]
    breed_name = dog_names[class_number].split('.')[1]
    #print('The file "',file_name, '" is class number #', class_number,':',breed_name)
    unordered_dict[class_number]=breed_name
    
breed_dict = dict(sorted(unordered_dict.items()))
breed_dict

{0: 'Affenpinscher',
 1: 'Afghan_hound',
 2: 'Airedale_terrier',
 3: 'Akita',
 4: 'Alaskan_malamute',
 5: 'American_eskimo_dog',
 6: 'American_foxhound',
 7: 'American_staffordshire_terrier',
 8: 'American_water_spaniel',
 9: 'Anatolian_shepherd_dog',
 10: 'Australian_cattle_dog',
 11: 'Australian_shepherd',
 12: 'Australian_terrier',
 13: 'Basenji',
 14: 'Basset_hound',
 15: 'Beagle',
 16: 'Bearded_collie',
 17: 'Beauceron',
 18: 'Bedlington_terrier',
 19: 'Belgian_malinois',
 20: 'Belgian_sheepdog',
 21: 'Belgian_tervuren',
 22: 'Bernese_mountain_dog',
 23: 'Bichon_frise',
 24: 'Black_and_tan_coonhound',
 25: 'Black_russian_terrier',
 26: 'Bloodhound',
 27: 'Bluetick_coonhound',
 28: 'Border_collie',
 29: 'Border_terrier',
 30: 'Borzoi',
 31: 'Boston_terrier',
 32: 'Bouvier_des_flandres',
 33: 'Boxer',
 34: 'Boykin_spaniel',
 35: 'Briard',
 36: 'Brittany',
 37: 'Brussels_griffon',
 38: 'Bull_terrier',
 39: 'Bulldog',
 40: 'Bullmastiff',
 41: 'Cairn_terrier',
 42: 'Canaan_dog',
 43: '

In [9]:
# save breed dictionary together with the model
import csv
with open('app/breed_dict.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in breed_dict.items():
       writer.writerow([key, value])

## Loading the trained model

In [47]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()
num_classes = len(breed_dict) #133

### Architecture definition.
input_size = [img_tensor_normalized.shape[1],img_tensor_normalized.shape[2],img_tensor_normalized.shape[3]]
# 1st layer convulotional is the imput layer with the size of the images
model.add(Conv2D(filters=16,kernel_size=2, strides=1, activation='relu', input_shape= input_size))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2, strides=1,activation='relu', input_shape= input_size))
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
model.add(Conv2D(filters=64,kernel_size=2, strides=1,activation='relu', input_shape= input_size))
model.add(MaxPooling2D(pool_size=2))
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax')) # we use activation softmax for clasification problem.

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 223, 223, 16)      208       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 110, 110, 32)      2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 54, 54, 64)        8256      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
global_average_pooling2d_2 ( (None, 64)                0         
__________

In [48]:
# compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [49]:
#Load the Model with the Best Validation Loss
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

In [ ]:
# Load breed dictionary
with open('app/breed_dict.csv') as csv_file:
    reader = csv.reader(csv_file)
    breed_dict = dict(reader)

## Execute prediciton

In [52]:
from keras.preprocessing import image                  
from tqdm import tqdm

In [56]:
def breed_prediction(img_path, model):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    img_tensor = np.expand_dims(x, axis=0)
    img_tensor_normalized = img_tensor.astype('float32')/255
    # Get index of predicted dog breed for each image in test set
    dog_breed_prediction = np.argmax(model.predict(img_tensor_normalized))
    print('The breed of the given picture is likely a dog breed "', breed_dict[str(dog_breed_prediction)], '"')

### Test the Model

Try out your model on the test dataset of dog images.  Ensure that your test accuracy is greater than 1%.

In [64]:
#img_path = '../../../data/dog_images/test/001.Affenpinscher/Affenpinscher_00058.jpg'
#img_path = '../../../data/dog_images/test/009.American_water_spaniel/American_water_spaniel_00631.jpg'
#img_path = '../../../data/dog_images/test/015.Basset_hound/Basset_hound_01097.jpg'
#img_path = '../../../data/dog_images/test/042.Cairn_terrier/Cairn_terrier_02979.jpg'
img_path = '../../../data/dog_images/test/078.Great_dane/Great_dane_05329.jpg'
#img_path = '../../../data/dog_images/test/057.Dalmatian/Dalmatian_04056.jpg'

image_name=img_path.split('/')[-1]

breed_prediction(img_path, model)

The breed of the given picture is likely a dog breed " Boston_terrier "
